In [1]:
import json
import sys
import re
from nltk import word_tokenize

from os import listdir
from bz2 import BZ2File as bzopen
from time import time
from collections import Counter
import pandas as pd



In [2]:
# locate comment files and thread files
comment_dir = '../data/'
comment_files = [f for f in listdir(comment_dir) if '.bz2' in f]
unique_months = [f[:f.index('.')] for f in comment_files]

print(unique_months)

['2017-01']


In [4]:
print('\nVariable initializing...')
comments = {'comment_id':[], 'length': [], 'month': [], 'subreddit': []}
temp_threads = {} # {thread_id: {'body': [comment1, comment2], 'month':[month1, month2], 'users': [user1, user2...] }}
threads = {'thread_id': [], 'length':[], 'num_user':[], 'num_comment':[]}

temp_users = {} # {user_id: {month: {'thread_id':[], 'num_comment':0, 'subreddit': []}}}
users = {'user_id': []}

temp_subreddit = {} # {subreddit: {month: {'user':[], 'thread':[], 'num_comment': 0}}}
subreddit = {'subreddit': []}

for m in unique_months:
	threads[m] = []

	users[m+'_num_thread'] = []
	users[m+'_num_comment'] = []
	users[m+'_num_subreddit'] = []

	subreddit[m+'_num_thread'] = []
	subreddit[m+'_num_comment'] = []
	subreddit[m+'_num_user'] = []
	subreddit[m+'_num_unique_user'] = []





Variable initializing...


In [5]:
for f in comment_files:
	with bzopen(comment_dir + f, 'rb') as bzfin:
		month = f[:f.index('.')]
		print("\nStart working on month:", month)

		i = 0
		for line in bzfin.readlines():
			line = line.decode('utf-8')
			line = json.loads(line)

			if i % 10000 == 0:
				print('Start extracting attributes of COMMENTS (comments))')
			# comments attributes
			comment_id = line['id']
			comment = line['body']
			tokens = word_tokenize(comment)
			subreddit = line['subreddit']

			comments['comment_id'].append(comment_id)
			comments['length'].append(len(tokens))
			comments['month'].append(month)
			comments['subreddit'].append(subreddit)

			if i % 10000 == 0:
				print('Start extracting attributes of THREADS (temp_threads)')
			# threads attributes
			thread_id = line['link_id']
			if thread_id not in temp_threads:
				temp_threads[thread_id] = {'body': [], 'month':[], 'user': []}
			user = line['author']
			temp_threads[thread_id]['body'].append(comment)
			temp_threads[thread_id]['month'].append(month)
			temp_threads[thread_id]['user'].append(user)

			if i % 10000 == 0:
				print('Start extracting attributes of USER (temp_users)')
			if user not in temp_users:
				temp_users[user] = {}
			if month not in temp_users[user]:
				temp_users[user][month] = {'thread_id':[], 'num_comment':0, 'subreddit': []}

			user_month_dict = temp_users[user][month]
			if thread_id not in user_month_dict['thread_id']:
				user_month_dict['thread_id'].append(thread_id)
			if subreddit not in user_month_dict['subreddit']:
				user_month_dict['subreddit'].append(subreddit)
			user_month_dict['num_comment'] += 1

			if i % 10000 == 0:
				print('Start extracting attributes of SUBREDDIT (temp_subreddit)')
			if subreddit not in temp_subreddit:
				temp_subreddit[subreddit] = {}
			if month not in temp_subreddit[subreddit]:
				temp_subreddit[subreddit][month] = {'user':[], 'thread':[], 'num_comment': 0}

			subreddit_month = temp_subreddit[subreddit][month]
			subreddit_month['user'].append(user)
			if thread_id not in subreddit_month['thread']:
				subreddit_month['thread'].append(thread_id)
			subreddit_month['num_comment'] += 1

			i += 1


Start working on month: 2017-01
Start extracting attributes of COMMENTS (comments))
Start extracting attributes of THREADS (temp_threads)
Start extracting attributes of USER (temp_users)
Start extracting attributes of SUBREDDIT (temp_subreddit)
Start extracting attributes of COMMENTS (comments))
Start extracting attributes of THREADS (temp_threads)
Start extracting attributes of USER (temp_users)
Start extracting attributes of SUBREDDIT (temp_subreddit)
Start extracting attributes of COMMENTS (comments))
Start extracting attributes of THREADS (temp_threads)
Start extracting attributes of USER (temp_users)
Start extracting attributes of SUBREDDIT (temp_subreddit)
Start extracting attributes of COMMENTS (comments))
Start extracting attributes of THREADS (temp_threads)
Start extracting attributes of USER (temp_users)
Start extracting attributes of SUBREDDIT (temp_subreddit)
Start extracting attributes of COMMENTS (comments))
Start extracting attributes of THREADS (temp_threads)
Start ext

In [6]:
print('\n1. Assembling COMMENTS DataFrame (comments_df)...')
comments_df = pd.DataFrame(comments)


1. Assembling COMMENTS DataFrame (comments_df)...


In [8]:
comments_df

,comment_id,length,month,subreddit
0,dbumnvn,9,2017-01,CryptoCurrency
1,dbumo5b,22,2017-01,Bitcoin
2,dbumpch,6,2017-01,Bitcoin
3,dbumpkm,66,2017-01,BitcoinMarkets
4,dbumpri,3,2017-01,btc
5,dbumpsl,19,2017-01,Bitcoin
6,dbumsdc,28,2017-01,BitcoinMarkets
7,dbumsmd,142,2017-01,Bitcoin
8,dbumuee,10,2017-01,btc
9,dbumukv,27,2017-01,Bitcoin


In [19]:
print('2. Assembling THREADS DataFrame (threads_df)...')
threads = {'thread_id': [], 'length':[], 'num_user':[], 'num_comment':[]}
for m in unique_months:
    threads[m] = []
    
i = 0
for thread_id, content in temp_threads.items():

    # comments related attributes
    body = content['body']
    length = 0
    for comment in body:
        tokens = word_tokenize(comment)
        length += len(tokens)

    # month 
    month = content['month']
    month_count = Counter(month)


    threads['thread_id'].append(thread_id)
    threads['length'].append(length)
    threads['num_user'].append(len(content['user']))
    threads['num_comment'].append(len(body))
    len_thread_id = len(threads['thread_id'])
    len_length = len(threads['length'])
    len_num_user = len(threads['num_user'])
    len_num_comment = len(threads['num_comment'])
    
    for m in unique_months:
        count = month_count.get(m, 0)
        threads[m].append(count)
        len1 = len(threads[m])
    if len_thread_id != len_num_user:
        print(i, thread_id)
        
    i += 1
        
# threads_df = pd.DataFrame(threads)

2. Assembling THREADS DataFrame (threads_df)...


In [20]:
for k in threads:
    print(k, len(threads[k]))

thread_id 9690
length 9690
num_user 9690
num_comment 9690
2017-01 9690


In [21]:
threads_df = pd.DataFrame(threads)

In [22]:
threads_df

,2017-01,length,num_comment,num_user,thread_id
0,1,9,1,1,t3_5l72cl
1,9,255,9,9,t3_5lcdp9
2,22,860,22,22,t3_5l9geh
3,55,1511,55,55,t3_5l7tm2
4,6,106,6,6,t3_5l9hho
5,23,796,23,23,t3_5lcc9o
6,40,1277,40,40,t3_5lc7av
7,50,9473,50,50,t3_5l66a7
8,1,10,1,1,t3_5lantg
9,3,79,3,3,t3_5lal7d


In [24]:
print('3. Assembling USERS DataFrame (user_df)...')
for user_id, month_content in temp_users.items():
	users['user_id'].append(user_id)
	for month, content in month_content.items():
		num_thread = len(content['thread_id'])
		num_subreddit = len(content['subreddit'])
		users[month+'_num_thread'].append(num_thread)
		users[month+'_num_comment'].append(content['num_comment'])
		users[month+'_num_subreddit'].append(num_subreddit)
	
user_df = pd.DataFrame(users)

3. Assembling USERS DataFrame (user_df)...


In [23]:
users

{'2017-01_num_comment': [],
 '2017-01_num_subreddit': [],
 '2017-01_num_thread': [],
 'user_id': []}

In [25]:
user_df

,2017-01_num_comment,2017-01_num_subreddit,2017-01_num_thread,user_id
0,13,3,12,twigwam
1,146,1,99,gulfbitcoin
2,795,4,220,Full_Node
3,53,3,16,Jeff_R
4,11377,8,3626,[deleted]
5,322,3,84,thisusernamelovesyou
6,112,2,65,1BitcoinOrBust
7,85,2,65,Sugar_Daddy_Peter
8,64,2,46,Jonnyb42
9,18,1,12,Codebender


In [31]:
subreddit = {'subreddit': []}

for m in unique_months:
	threads[m] = []

	users[m+'_num_thread'] = []
	users[m+'_num_comment'] = []
	users[m+'_num_subreddit'] = []

	subreddit[m+'_num_thread'] = []
	subreddit[m+'_num_comment'] = []
	subreddit[m+'_num_user'] = []
	subreddit[m+'_num_unique_user'] = []


print('4. Assembling SUBREDDIT DataFrame (subreddit_df)...')
for subreddit_name, month_content in temp_subreddit.items():
	subreddit['subreddit'].append(subreddit_name)
	for month, content in month_content.items():
		num_thread = len(content['thread'])
		num_comment = content['num_comment']
		num_user = len(content['user'])
		num_unique_user = len(set(content['user']))


		subreddit[month+'_num_thread'].append(num_thread)
		subreddit[month+'_num_comment'].append(num_comment)
		subreddit[month+'_num_user'].append(num_user)
		subreddit[month+'_num_unique_user'].append(num_unique_user)
subreddit_df = pd.DataFrame(subreddit)

4. Assembling SUBREDDIT DataFrame (subreddit_df)...


In [33]:
subreddit_df

,2017-01_num_comment,2017-01_num_thread,2017-01_num_unique_user,2017-01_num_user,subreddit
0,2215,449,683,2215,CryptoCurrency
1,86271,6104,10077,86271,Bitcoin
2,26460,418,2027,26460,BitcoinMarkets
3,30025,1689,2333,30025,btc
4,11447,898,1189,11447,Monero
5,119,31,59,119,CryptoMarkets
6,102,56,54,102,BlockChain
7,211,45,74,211,Ripple


In [34]:
subreddit_df.to_csv('subreddit_eda_data.tsv', index=False, sep='\t')